## Gripe

### Biblioteca

In [75]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import ipywidgets as widgets
from IPython.display import display
import ipywidgets as widgets
import numpy as np
import pickle
import os

### Coleta e tratamento dos dados

In [46]:

dados_2021 = pd.read_csv('dados/gripe_2021.csv', sep=';', encoding='ISO-8859-1')
dados_2022 = pd.read_csv('dados/gripe_2022.csv', sep=';', encoding='ISO-8859-1')
dados_2023 = pd.read_csv('dados/gripe_2023.csv', sep=';', encoding='ISO-8859-1')

dados = pd.concat([dados_2021, dados_2022, dados_2023])


/var/folders/z3/2x5_0s3j31zdf3c9g85v3rbc0000gn/T/ipykernel_21832/152469830.py:1: DtypeWarning: Columns (15,59,61,62,63,91,93,105,107,114,116,117,118,122,143) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_2021 = pd.read_csv('dados/gripe_2021.csv', sep=';', encoding='ISO-8859-1')
/var/folders/z3/2x5_0s3j31zdf3c9g85v3rbc0000gn/T/ipykernel_21832/152469830.py:2: DtypeWarning: Columns (15,53,59,61,62,63,91,93,120,122,143,171) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_2022 = pd.read_csv('dados/gripe_2022.csv', sep=';', encoding='ISO-8859-1')
/var/folders/z3/2x5_0s3j31zdf3c9g85v3rbc0000gn/T/ipykernel_21832/152469830.py:3: DtypeWarning: Columns (15,18,20,23,53,59,61,63,72,74,79,91,93,109,130,175,178) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_2023 = pd.read_csv('dados/gripe_2023.csv', sep=';', encoding='ISO-8859-1')


In [47]:
dados.columns

Index(['DT_NOTIFIC', 'SEM_NOT', 'DT_SIN_PRI', 'SEM_PRI', 'SG_UF_NOT',
       'ID_REGIONA', 'CO_REGIONA', 'ID_MUNICIP', 'CO_MUN_NOT', 'ID_UNIDADE',
       ...
       'VG_ENC', 'VG_REINF', 'REINF', 'FAB_ADIC', 'LOT_RE_BI', 'FAB_RE_BI',
       'DOSE_ADIC', 'DOS_RE_BI', 'LOTE_ADIC', 'TABAG'],
      dtype='object', length=190)

In [48]:
# Verificando nulos e inválidos
dados.isna().sum()

DT_NOTIFIC          0
SEM_NOT             0
DT_SIN_PRI          0
SEM_PRI             0
SG_UF_NOT           0
               ...   
FAB_RE_BI     2545509
DOSE_ADIC     2561067
DOS_RE_BI     2545495
LOTE_ADIC     2561129
TABAG         2566070
Length: 190, dtype: int64

In [49]:
# Concatenando os dados
dados = pd.concat([dados_2021, dados_2022, dados_2023])

# Mantendo apenas as colunas relevantes
dados = dados[['DT_NOTIFIC', 'SG_UF_NOT', 'CO_REGIONA', 'CO_MUN_NOT', 'CS_SEXO', 'DT_NASC', 'NU_IDADE_N', 'CS_GESTANT', 'CS_RACA']]

# Ajustando a conversão de datas
dados['DT_NOTIFIC'] = pd.to_datetime(dados['DT_NOTIFIC'], format='%d/%m/%Y')
dados['DT_NASC'] = pd.to_datetime(dados['DT_NASC'], format='%d/%m/%Y', errors='coerce')  # usando errors='coerce' para lidar com qualquer dado incorreto

# Criando metadados
dados['ANO'] = dados['DT_NOTIFIC'].dt.year
dados['MES'] = dados['DT_NOTIFIC'].dt.month

# Convertendo variáveis categóricas em numéricas conforme necessário
dados = pd.get_dummies(dados, columns=['SG_UF_NOT', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA'])

In [50]:
# Removendo linhas onde CO_REGIONA é nulo
dados = dados.dropna(subset=['CO_REGIONA'])
dados = dados.dropna(subset=['DT_NASC'])
print(dados.isnull().sum())

DT_NOTIFIC      0
CO_REGIONA      0
CO_MUN_NOT      0
DT_NASC         0
NU_IDADE_N      0
ANO             0
MES             0
SG_UF_NOT_AC    0
SG_UF_NOT_AL    0
SG_UF_NOT_AM    0
SG_UF_NOT_AP    0
SG_UF_NOT_BA    0
SG_UF_NOT_CE    0
SG_UF_NOT_DF    0
SG_UF_NOT_ES    0
SG_UF_NOT_GO    0
SG_UF_NOT_MA    0
SG_UF_NOT_MG    0
SG_UF_NOT_MS    0
SG_UF_NOT_MT    0
SG_UF_NOT_PA    0
SG_UF_NOT_PB    0
SG_UF_NOT_PE    0
SG_UF_NOT_PI    0
SG_UF_NOT_PR    0
SG_UF_NOT_RJ    0
SG_UF_NOT_RN    0
SG_UF_NOT_RO    0
SG_UF_NOT_RR    0
SG_UF_NOT_RS    0
SG_UF_NOT_SC    0
SG_UF_NOT_SE    0
SG_UF_NOT_SP    0
SG_UF_NOT_TO    0
CS_SEXO_1       0
CS_SEXO_F       0
CS_SEXO_I       0
CS_SEXO_M       0
CS_GESTANT_0    0
CS_GESTANT_1    0
CS_GESTANT_2    0
CS_GESTANT_3    0
CS_GESTANT_4    0
CS_GESTANT_5    0
CS_GESTANT_6    0
CS_GESTANT_9    0
CS_RACA_1.0     0
CS_RACA_2.0     0
CS_RACA_3.0     0
CS_RACA_4.0     0
CS_RACA_5.0     0
CS_RACA_9.0     0
dtype: int64


### Separando previsores e alvo

In [51]:

dados_agregados = dados.groupby(['ANO', 'MES', 'CO_REGIONA']).size().reset_index(name='CASOS')
previsores = dados_agregados.drop('CASOS', axis=1)
alvo = dados_agregados['CASOS']
previsores_arquivo = os.path.join('./variaveis', 'previsores.pickle')
with open(previsores_arquivo, 'wb') as f:
    pickle.dump(previsores, f)
print(f'Previsores salvos em {previsores_arquivo}')

alvo_arquivo = os.path.join('./variaveis', 'alvo.pickle')
with open(alvo_arquivo, 'wb') as f:
    pickle.dump(alvo, f)
print(f'Alvo salvo em {alvo_arquivo}')

Previsores salvos em ./variaveis/previsores.pickle
Alvo salvo em ./variaveis/alvo.pickle


### Escalonando

In [52]:
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

### Divisão Treino Teste 70 - 30

In [60]:

previsores_arquivo = './variaveis/previsores.pickle'
with open(previsores_arquivo, 'rb') as f:
    previsores = pickle.load(f)
print(f'Previsores carregados de {previsores_arquivo}')

alvo_arquivo = './variaveis/alvo.pickle'
with open(alvo_arquivo, 'rb') as f:
    alvo = pickle.load(f)
print(f'Alvo carregado de {alvo_arquivo}')



Previsores carregados de ./variaveis/previsores.pickle
Alvo carregado de ./variaveis/alvo.pickle


In [61]:
X_train, X_test, y_train, y_test = train_test_split(previsores, alvo, test_size=0.3, random_state=42)

### Loop algoritimo

In [64]:

modelos = {
    #'RandomForest': RandomForestClassifier(),
    #'GradientBoosting': GradientBoostingClassifier(),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(max_iter=50) 
}

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    predicoes = modelo.predict(X_test)
    
    acuracia = accuracy_score(y_test, predicoes)
    f1 = f1_score(y_test, predicoes, average='macro')
    matriz_confusao = confusion_matrix(y_test, predicoes)
    
    print(f'{nome} Acurácia: {acuracia}')
    print(f'{nome} F1 Score: {f1}')
    print(f'{nome} Matriz de Confusão:\n{matriz_confusao}')
    
    # Validação cruzada para acurácia
    scores = cross_val_score(modelo, X_train, y_train, scoring='accuracy', cv=5)
    print(f'{nome} Acurácia Média de Validação Cruzada: {scores.mean()}\n')
    
    # Salvar o modelo em arquivo pickle
    modelo_arquivo = os.path.join('./modelos', f'{nome}.pickle')
    with open(modelo_arquivo, 'wb') as f:
        pickle.dump(modelo, f)
    print(f'Modelo {nome} salvo em {modelo_arquivo}\n')



KNN Acurácia: 0.04554455445544554
KNN F1 Score: 0.0016662131455732396
KNN Matriz de Confusão:
[[75 18  7 ...  0  0  0]
 [39 18  5 ...  0  0  0]
 [28 14 11 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KNN Acurácia Média de Validação Cruzada: 0.04498508995576536

Modelo KNN salvo em ./modelos/KNN.pickle



/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


MLP Acurácia: 0.03498349834983498
MLP F1 Score: 0.0003312170921924841
MLP Matriz de Confusão:
[[58 61 10 ...  0  0  0]
 [37 42  5 ...  0  0  0]
 [31 51  5 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Acurácia Média de Validação Cruzada: 0.0226351388551048

Modelo MLP salvo em ./modelos/MLP.pickle



/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


### Criação de hibirdos

In [65]:

# Caminhos dos arquivos dos modelos salvos
algo_1_path = './modelos/RandomForest.pickle'
algo_2_path = './modelos/MLP.pickle'

# Carregando os modelos previamente treinados
with open(algo_1_path, 'rb') as f:
    algo_1 = pickle.load(f)
print(f'Modelo RandomForest carregado de {algo_1_path}')

with open(algo_2_path, 'rb') as f:
    algo_2 = pickle.load(f)
print(f'Modelo MLP carregado de {algo_2_path}')

# Criando um modelo de votação híbrido usando os classificadores carregados
voting_clf = VotingClassifier(estimators=[('rf', algo_1), ('gb', algo_2)], voting='soft')
voting_clf.fit(X_train, y_train)  # Treinando o classificador de votação com os dados

# Previsões e métricas para o modelo híbrido
pred_hibrido = voting_clf.predict(X_test)

# Calculando métricas para o modelo híbrido
acuracia_hibrido = accuracy_score(y_test, pred_hibrido)
f1_hibrido = f1_score(y_test, pred_hibrido, average='macro')
matriz_confusao_hibrido = confusion_matrix(y_test, pred_hibrido)

print(f'Modelo Híbrido Acurácia: {acuracia_hibrido}')
print(f'Modelo Híbrido F1 Score: {f1_hibrido}')
print(f'Modelo Híbrido Matriz de Confusão:\n{matriz_confusao_hibrido}')

# Salvando o modelo híbrido em um arquivo pickle
modelo_hibrido_arquivo = os.path.join('./modelos', 'ModeloHibrido.pickle')
with open(modelo_hibrido_arquivo, 'wb') as f:
    pickle.dump(voting_clf, f)
print(f'Modelo híbrido salvo em {modelo_hibrido_arquivo}')

Modelo RandomForest carregado de ./modelos/RandomForest.pickle
Modelo MLP carregado de ./modelos/MLP.pickle


/Users/rafaelmunareto/anaconda3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


Modelo Híbrido Acurácia: 0.02145214521452145
Modelo Híbrido F1 Score: 0.0011988465934886997
Modelo Híbrido Matriz de Confusão:
[[26 12  7 ...  0  0  0]
 [10  9  2 ...  0  0  0]
 [10  2  3 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
Modelo híbrido salvo em ./modelos/ModeloHibrido.pickle


### Resultado

In [74]:
ano_widget = widgets.IntSlider(value=2024, min=2019, max=2024, step=1, description='Ano:')
mes_widget = widgets.Dropdown(options=list(range(1, 13)), value=2, description='Mês:')
regioes_disponiveis = sorted(dados['CO_REGIONA'].unique())
regiao_widget = widgets.Dropdown(options=regioes_disponiveis, value=regioes_disponiveis[0], description='Região:')

display(ano_widget, mes_widget, regiao_widget)

# Função para realizar predições
# Removendo a referência ao OneHotEncoder e ajustando a predição
def realizar_predicoes(ANO, MES, REGIAO):
    predicoes_modelos = []
    diretorio_modelos = './modelos'

    # Ajustando para o uso sem necessidade de aplicação do OneHotEncoder novamente
    for arquivo_modelo in os.listdir(diretorio_modelos):
        caminho_completo = os.path.join(diretorio_modelos, arquivo_modelo)
        
        with open(caminho_completo, 'rb') as f:
            modelo = pickle.load(f)

        # Supondo que 'REGIAO' seja numérica e esteja disponível diretamente sem a necessidade de dummies
        entrada_predicao = pd.DataFrame({'ANO': [ANO], 'MES': [MES], 'REGIAO': [REGIAO]})
        predicao = modelo.predict(entrada_predicao)
        predicoes_modelos.append(predicao[0])
    
    pred_media = np.mean(predicoes_modelos)
    print(f"\nPredição média de casos para a região {REGIAO} no mês {MES} de {ANO}: {pred_media}")

    # Comparação com ano anterior
    try:
        casos_ano_anterior = dados[(dados['ANO'] == ANO - 1) & (dados['MES'] == MES) & (dados['REGIAO'] == REGIAO)]['CASOS'].values[0]
        aumento_percentual = ((pred_media - casos_ano_anterior) / casos_ano_anterior) * 100
        print(f"Aumento percentual de casos em relação ao mesmo período do ano anterior: {aumento_percentual}%")
    except IndexError:
        print("Dados para o ano anterior não encontrados.")


# Botão para realizar predições
predicao_botao = widgets.Button(description="Realizar Predição")
predicao_output = widgets.Output()

def on_predicao_botao_clicked(b):
    with predicao_output:
        print("Calculando predições...")
        realizar_predicoes(ano_widget.value, mes_widget.value, regiao_widget.value)

predicao_botao.on_click(on_predicao_botao_clicked)

display(predicao_botao, predicao_output)


IntSlider(value=2024, description='Ano:', max=2024, min=2019)

Dropdown(description='Mês:', index=1, options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=2)

Dropdown(description='Região:', options=(1331.0, 1332.0, 1333.0, 1334.0, 1335.0, 1336.0, 1337.0, 1338.0, 1339.…

Button(description='Realizar Predição', style=ButtonStyle())

Output()